In [193]:
import pandas as pd
import json

In [194]:
file_path = "/Users/Lucas/Desktop/Dev/Webapp/data/DI_CONNECT/DI-Connect-Fitness/lucas.hagelstein@gmail.com_0_summarizedActivities.json"


In [195]:
with open(file_path, "r") as f:
    data = json.load(f)

f.close

<function TextIOWrapper.close()>

In [196]:
activities_json = data[0]["summarizedActivitiesExport"]

In [197]:
df = pd.json_normalize(activities_json)

#### Transform durations

In [198]:
def convert_ms_to_min(x):
    return x / 60000

In [200]:
cols = [
    'duration', 
    'elapsedDuration', 
    'movingDuration', 
    'hrTimeInZone_0', 
    'hrTimeInZone_1', 
    'hrTimeInZone_2', 
    'hrTimeInZone_3', 
    'hrTimeInZone_4', 
    'hrTimeInZone_5', 
    'powerTimeInZone_0', 
    'powerTimeInZone_1', 
    'powerTimeInZone_2', 
    'powerTimeInZone_3', 
    'powerTimeInZone_4', 
    'powerTimeInZone_5', 
    'powerTimeInZone_6', 
    'powerTimeInZone_7'
    ]

for col in cols:
    df[col] = df[col].apply(convert_ms_to_min)

#### Timestamps

In [201]:
cols = [
    'beginTimestamp',
    'startTimeGmt',
    'startTimeLocal',
    'firstbeatData.results.timestamp'
]

for col in cols:
    df[col] = pd.to_datetime(df[col], unit='ms')

#### Distances

In [202]:
df['distance'] = df['distance'] / 100000


In [203]:
cols = [
    'elevationGain',
    'elevationLoss',
    'minElevation',
    'maxElevation'
]

for col in cols:
    df[col] = df[col] / 100

#### Speed

In [204]:
cols = [
    'avgSpeed',
    'maxSpeed',
    'maxVerticalSpeed'
]

for col in cols:
    df[col] = df[col] * 36

#### Calories

In [205]:
df['calories'] = df['calories'] / 4.184

### Split Dataframe into Activities

In [206]:
activities = df['activityType'].unique()

In [207]:
activities_dict = {}
activity_threshold = 10
for i in activities:
    if len(df[df['activityType'] == i]) > activity_threshold:
        print(f'Create Dataframe: {i}_df')
        activities_dict[f'{i}_df'] = df[df['activityType'] == i]
    else:
        print(f'Dataframe rejected: {i} (less than {activity_threshold} activities)')

Create Dataframe: virtual_ride_df
Dataframe rejected: paddelball (less than 10 activities)
Create Dataframe: strength_training_df
Create Dataframe: running_df
Create Dataframe: walking_df
Dataframe rejected: hiit (less than 10 activities)
Dataframe rejected: lap_swimming (less than 10 activities)
Dataframe rejected: swimming (less than 10 activities)
Dataframe rejected: breathwork (less than 10 activities)
Create Dataframe: other_df
Dataframe rejected: indoor_cycling (less than 10 activities)
Dataframe rejected: road_biking (less than 10 activities)
Create Dataframe: indoor_cardio_df
Dataframe rejected: cycling (less than 10 activities)


In [208]:
activities_dict['running_df'].drop(columns=['splitSummaries', 'splits'], inplace=True)

/var/folders/_6/ptdbckg91p3bnswxsg_mt8pw0000gp/T/ipykernel_88384/1569287344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_dict['running_df'].drop(columns=['splitSummaries', 'splits'], inplace=True)


In [209]:
activities_dict['running_df']

,activityId,uuidMsb,uuidLsb,name,activityType,userProfileId,timeZoneId,beginTimestamp,eventTypeId,rule,sportType,startTimeGmt,startTimeLocal,duration,distance,elevationGain,elevationLoss,avgSpeed,maxSpeed,avgHr,maxHr,minHr,avgPower,maxPower,avgBikeCadence,maxBikeCadence,calories,startLongitude,startLatitude,aerobicTrainingEffect,strokes,normPower,max20MinPower,avgFractionalCadence,maxFractionalCadence,elapsedDuration,movingDuration,anaerobicTrainingEffect,deviceId,minElevation,maxElevation,maxVerticalSpeed,manufacturer,lapCount,endLongitude,endLatitude,trainingEffectLabel,activityTrainingLoad,aerobicTrainingEffectMessage,anaerobicTrainingEffectMessage,maxLatitude,maxLongitude,minLatitude,minLongitude,hrTimeInZone_0,hrTimeInZone_1,hrTimeInZone_2,hrTimeInZone_3,hrTimeInZone_4,hrTimeInZone_5,powerTimeInZone_0,powerTimeInZone_1,powerTimeInZone_2,powerTimeInZone_3,powerTimeInZone_4,powerTimeInZone_5,powerTimeInZone_6,powerTimeInZone_7,decoDive,purposeful,autoCalcCalories,favorite,pr,elevationCorrected,atpActivity,parent,firstbeatData.results.timestamp,firstbeatData.results.sport,firstbeatData.results.subSport,firstbeatData.results.maxMet,firstbeatData.results.maxMetHeat,firstbeatData.results.maxMetHeatAlti,firstbeatData.results.maxMetUncorrected,firstbeatData.results.maxMetMins,firstbeatData.results.maximalBodyStatus,firstbeatData.results.recoveryTimeMins,firstbeatData.results.stressBalance,firstbeatData.results.trainingLoadPeakSumSession,firstbeatData.results.trainingLoadPeakAnaerobicSession,firstbeatData.results.aerobicTrainingEffect,firstbeatData.results.anaerobicTrainingEffect,firstbeatData.results.aerobicTrainingEffectDescIndex,firstbeatData.results.anaerobicTrainingEffectDescIndex,firstbeatData.results.workoutLabel,firstbeatData.results.eteVersion,firstbeatData.results.correctedHr,firstbeatData.results.maximalHr,firstbeatData.results.energyExpenditure,firstbeatData.results.cumulativeEnergyExpenditure,avgRunCadence,maxRunCadence,steps,bmrCalories,avgStrideLength,avgDoubleCadence,maxDoubleCadence,waterEstimated,moderateIntensityMinutes,vigorousIntensityMinutes,differenceBodyBattery,hrTimeInZone_6,summarizedExerciseSets,activeSets,totalSets,totalReps,workoutId,avgVerticalOscillation,avgGroundContactTime,vO2MaxValue,avgVerticalRatio,locationName,minRespirationRate,maxRespirationRate,avgRespirationRate,avgGradeAdjustedSpeed,isRunPowerWindDataEnabled,workoutFeel,workoutRpe,workoutComplianceScore,runPowerWindDataEnabled,avgStrokes,activeLengths,avgSwolf,poolLength,avgStrokeDistance,avgSwimCadence,surfaceInterval,avgVerticalSpeed,floorsClimbed,floorsDescended,jumpCount,startStress,endStress,differenceStress,avgStress,maxStress,summarizedDiveInfo.totalSurfaceTime,perceivedWorkoutEffort
6,18194527149,2101793468932310615,-7388135326219629078,Böblingen - z2 - 30min,running,35889732,124,2025-02-05 10:29:33.000,9,public,RUNNING,2025-02-05 10:29:33,2025-02-05 11:29:33,31.527016,5.00718,3.460000,2.160000,9.5292,11.253600,142.0,150.0,83.0,308.0,378.0,NaN,NaN,401.576965,9.020577,48.692469,3.0,NaN,312.0,NaN,0.242188,0.0,31.527016,31.505317,0.0,3455326928,464.000000,467.600006,1.440088,GARMIN,6.0,9.021137,48.692651,AEROBIC_BASE,71.380707,IMPROVING_AEROBIC_BASE_8,NO_ANAEROBIC_BENEFIT_0,48.694106,9.024354,48.692469,9.020577,0.278917,1.850050,12.584383,16.815167,0.0,0.0,1.495567,24.516667,4.232750,1.283533,0.0,0.0,NaN,NaN,False,False,False,False,False,False,False,False,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,118.0,5122.0,201.12096,97.490002,162.484375,236.0,453.0,1.0,30.0,-8.0,NaN,NaN,NaN,NaN,NaN,1.117346e+09,8.78,283.100006,49.0,9.02,Böblingen,14.960000,39.700001,31.340000,0.2649,True,100.0,30.0,83.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,18165324023,4778129481110736427,-7837198492406312548,Böblingen Laufen,running,35889732,124,2025-02-02 08:17:59.000,9,public,RUNNING,2025-02-02 08:17:59,2025-02-02 09:17:59,26.099701,3.34404,30.800000,26.830000,7.6860